In [222]:
import parselmouth
from parselmouth.praat import call, run_file
import glob
import pandas as pd
import numpy as np
import scipy
from scipy.stats import binom
from scipy.stats import ks_2samp
from scipy.stats import ttest_ind
import os
import librosa
from termcolor import colored

In [14]:
import os, re, sys, pprint, string, math, time, copy
from os.path import isfile, join
from os import listdir

In [15]:
def feature_of_pauses(m,p):
    sound=m
    sourcerun="myspsolution.praat"
    path=p
    try:
        objects= run_file(sourcerun, -20, 2, 0.3, "yes",sound,path, 80, 400, 0.01, capture_output=True)
        z1=str( objects[1]) # This will print the info from the textgrid object, and objects[1] is a parselmouth.Data object with a TextGrid inside
        z2=z1.strip().split()
        z3=int(z2[1]) # will be the integer number 10
        z4=float(z2[3]) # will be the floating point number 8.3
        print ('pauses is obtained')
        return z3
    except:
        z3=0
        print (colored('could not obtain pauses','red') )
    return;

In [16]:
def feature_rate_of_speech(m,p):
    sound=m
    sourcerun="myspsolution.praat"
    path=p
    try:
        objects= run_file(sourcerun, -20, 2, 0.3, "yes",sound,path, 80, 400, 0.01, capture_output=True)
        z1=str( objects[1]) # This will print the info from the textgrid object, and objects[1] is a parselmouth.Data object with a TextGrid inside
        z2=z1.strip().split()
        z3=int(z2[2]) # will be the integer number 10
        z4=float(z2[3]) # will be the floating point number 8.3
        print ('rate_of_speech is obtained' )
        return z3
    except:
        z3=0
        print (colored('could not obtain rate_of_speech','red') )

    return;

In [17]:
def feature_articulation_rate(m,p):
    sound=m
    sourcerun="myspsolution.praat"
    path=p
    try:
        objects= run_file(sourcerun, -20, 2, 0.3, "yes",sound,path, 80, 400, 0.01, capture_output=True)
        z1=str( objects[1]) # This will print the info from the textgrid object, and objects[1] is a parselmouth.Data object with a TextGrid inside
        z2=z1.strip().split()
        z3=int(z2[3]) # will be the integer number 10
        z4=float(z2[3]) # will be the floating point number 8.3
        print ('articulation_rate is obtained')
        return z3
    except:
        z3=0
        print (colored('could not obtain articulation_rate','red') )

    return;

In [18]:
def feature_f0_mean_praat(m,p):
    sound=m
    sourcerun="myspsolution.praat"
    path=p
    try:
        objects= run_file(sourcerun, -20, 2, 0.3, "yes",sound,path, 80, 400, 0.01, capture_output=True)
        z1=str( objects[1]) # This will print the info from the textgrid object, and objects[1] is a parselmouth.Data object with a TextGrid inside
        z2=z1.strip().split()
        z3=int(z2[3]) # will be the integer number 10
        z4=float(z2[7]) # will be the floating point number 8.3
        print ('f0_mean_praat is obtained' )
        return z4
    except:
        z4=0
        print (colored('could not obtain f0_mean_praa','red') )

    return;

In [19]:
def feature_rms(waveform):
    rmse = np.mean(librosa.feature.rms(waveform))
    print ('rms is calculated')
    return rmse

In [20]:
def feature_zcr(waveform):
    zcr = np.mean( librosa.feature.zero_crossing_rate(waveform))
    print ('zcr is calculated')
    return zcr

In [21]:
def feature_f0mean_librosa(waveform):
    f0, voiced_flag, voiced_probs = librosa.pyin(y=waveform,
                                             fmin=librosa.note_to_hz('C2'),
                                             fmax=librosa.note_to_hz('C7'))
    f = f0[~np.isnan(f0)]
    print ('f0mean_librosa is calculated')
    return np.mean(f)

In [22]:
def get_features(path,p):
# load an individual soundfile           
        waveform , sr = librosa.load(path,sr=48000)
        print(path[23:]+" is loaded" )   
    # compute features of soundfile
        pauses = feature_of_pauses(path,p)
        rate_of_speech = feature_rate_of_speech(path,p)
        articulation_rate = feature_articulation_rate(path,p)
        f0_mean_praat=feature_f0_mean_praat(path,p)
        f0_mean_librosa=feature_f0mean_librosa(waveform)
        rms=feature_rms(waveform)
        zcr=feature_zcr(waveform)
        feature_matrix=np.array([])
        # use np.hstack to stack our feature arrays horizontally to create a feature matrix
        feature_matrix = np.hstack((pauses, rate_of_speech, articulation_rate,f0_mean_praat,
                                    f0_mean_librosa,rms,zcr))
            
#         ,rms,zcr
        return feature_matrix

In [23]:
def load_data(path,p):
        X,y=[],[]
        for root, dirs, files in os.walk(path):
            for filename in files:
                file_path = os.path.join(root, filename)
                label=root[-1:]
                features = get_features(file_path,p)
                X.append(features)
                y.append(label)
                # '\r' + end='' results in printing over same line
                print(filename + 'is processed')
                print('====================================')
        # Return arrays to plug into sklearn's cross-validation algorithms
        return np.array(X), np.array(y)

In [24]:
features, lables = load_data('chunks/','data/')

AA713_Q12.wav is loaded
pauses is obtained
could not obtain rate_of_speech
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
AA713_Q12.wavis processed
AA713_Q24.wav is loaded
pauses is obtained
could not obtain rate_of_speech
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
AA713_Q24.wavis processed
AM058_Q16._0.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
AM058_Q16._0.wavis processed
AM058_Q16._1.wav is loaded
could not obtain pauses
could not obtain rate_of_speech
could not obtain articulation_rate
could not obtain f0_mean_praa
f0mean_librosa is calculated
rms is calculated
zcr is calculated
AM058_Q16._1.wavis processed
AM058_Q16._10.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate

could not obtain pauses
could not obtain rate_of_speech
could not obtain articulation_rate
could not obtain f0_mean_praa
f0mean_librosa is calculated
rms is calculated
zcr is calculated
BL551_Q22._3.wavis processed
BL551_Q3._0.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
BL551_Q3._0.wavis processed
BL551_Q3._1.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
BL551_Q3._1.wavis processed
JS827_Q14._0.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
JS827_Q14._0.wavis processed
JS827_Q14._1.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is 

f0mean_librosa is calculated
rms is calculated
zcr is calculated
JS827_Q18._22.wavis processed
JS827_Q18._3.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
JS827_Q18._3.wavis processed
JS827_Q18._4.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
JS827_Q18._4.wavis processed
JS827_Q18._5.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
JS827_Q18._5.wavis processed
JS827_Q18._6.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
JS827_Q18._6.wavis processed
JS827_Q18._7.

f0mean_librosa is calculated
rms is calculated
zcr is calculated
MM647_Q17._0.wavis processed
MM647_Q17._1.wav is loaded
could not obtain pauses
could not obtain rate_of_speech
could not obtain articulation_rate
could not obtain f0_mean_praa
f0mean_librosa is calculated
rms is calculated
zcr is calculated
MM647_Q17._1.wavis processed
MM647_Q17._2.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
MM647_Q17._2.wavis processed
MM647_Q17._3.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
MM647_Q17._3.wavis processed
MM647_Q17._4.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
MM647_Q17._4.wavis proce

pauses is obtained
could not obtain rate_of_speech
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
MP959_Q24.wavis processed
MS090_Q11._0.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
MS090_Q11._0.wavis processed
MS090_Q11._1.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
MS090_Q11._1.wavis processed
MS090_Q14._0.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
MS090_Q14._0.wavis processed
MS090_Q14._1.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0me

...

pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
JS827_Q8._6.wavis processed
JS827_Q9._0.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
JS827_Q9._0.wavis processed
JS827_Q9._1.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
JS827_Q9._1.wavis processed
JS827_Q9._10.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
JS827_Q9._10.wavis processed
JS827_Q9._11.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_libr

pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
KB361_Q11._6.wavis processed
KB361_Q11._7.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
KB361_Q11._7.wavis processed
KB361_Q11._8.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
KB361_Q11._8.wavis processed
KB361_Q11._9.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
KB361_Q11._9.wavis processed
KB361_Q16._0.wav is loaded
pauses is obtained
could not obtain rate_of_speech
articulation_rate is obtained
f0_mean_praat is obtained
f

pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
KB361_Q21._6.wavis processed
KB361_Q23._0.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
KB361_Q23._0.wavis processed
KB361_Q23._1.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
KB361_Q23._1.wavis processed
KB361_Q23._2.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
KB361_Q23._2.wavis processed
KB361_Q3._0.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_

f0mean_librosa is calculated
rms is calculated
zcr is calculated
KB361_Q9._3.wavis processed
KB361_Q9._4.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
KB361_Q9._4.wavis processed
KB361_Q9._5.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
KB361_Q9._5.wavis processed
KB361_Q9._6.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
KB361_Q9._6.wavis processed
KB361_Q9._7.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
KB361_Q9._7.wavis processed
KB361_Q9._8.wav is load

pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
KN241_Q7._3.wavis processed
KN241_Q7._4.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
KN241_Q7._4.wavis processed
KN241_Q7._5.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
KN241_Q7._5.wavis processed
KN241_Q7._6.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa is calculated
rms is calculated
zcr is calculated
KN241_Q7._6.wavis processed
KN241_Q7._7.wav is loaded
pauses is obtained
rate_of_speech is obtained
articulation_rate is obtained
f0_mean_praat is obtained
f0mean_librosa

In [25]:
waveform , sr = librosa.load('chunks/comfortablity_1\AA713_Q12.wav',sr=48000)

In [31]:
features

array([[0, None, 3, ..., 96.2519368628494, 0.002189298626035452,
        0.011829789862575301],
       [0, None, 3, ..., 90.47433854906737, 0.008050226606428623,
        0.01706096690188172],
       [0.0, 2.0, 4.0, ..., 140.58172442165667, 0.010045279748737812,
        0.027637313308599695],
       ...,
       [2.0, 4.0, 4.0, ..., 107.06429930193039, 0.0212356299161911,
        0.05797243626331811],
       [1.0, 4.0, 5.0, ..., 96.598500875219, 0.01753460429608822,
        0.06474148592085235],
       [1.0, 2.0, 4.0, ..., 125.01949119615801, 0.009194008074700832,
        0.0486066519216134]], dtype=object)

In [34]:
print(f'\nAudio samples represented: {features.shape[0]}')
print(f'Numerical features extracted per sample: {features.shape[1]}')
features_df = pd.DataFrame(features) # make it pretty for display
features_df.head()


Audio samples represented: 782
Numerical features extracted per sample: 7


,0,1,2,3,4,5,6
0,0,None,3,190.95,96.251937,0.002189,0.01183
1,0,None,3,171.85,90.474339,0.00805,0.017061
2,0.0,2.0,4.0,118.45,140.581724,0.010045,0.027637
3,None,None,None,None,182.458989,0.014163,0.054994
4,1.0,4.0,4.0,112.03,86.312089,0.018072,0.044742


In [35]:
labels_name=[]
labels_name.append('pauses')
labels_name.append('rate_of_speech')
labels_name.append('articulation_rate')
labels_name.append('f0_mean_praat')
labels_name.append('f0mean_librosa')
labels_name.append('rms')
labels_name.append('zcr')

dataset = pd.DataFrame(features)
dataset.columns=labels_name
dataset.to_csv('data_withoutindex.csv', index=False)
dataset.to_csv('data.csv', index=False)
dataset['label']=lables
dataset.to_csv('data_withlabels.csv', index=False)


In [36]:
none = features == None

In [49]:
none

array([[False,  True, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [56]:
rows_non =none.any(axis=1)

In [53]:
f= features

In [77]:
none_index = none.any(axis=1)

In [78]:
none_index

array([ True,  True, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False,  True, False, False,  True, False,
       False, False, False, False,  True, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
       False, False, False,  True,  True, False, False, False, False,
       False, False,  True,  True,  True, False, False, False,  True,
        True, False, False, False,  True, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False,

In [63]:
features =f[~none.any(axis=1), :]

In [64]:
features.shape[0]

694

In [79]:
lables1=lables[~none_index]

In [80]:
lables1.shape[0]

694

In [121]:
dataset = pd.DataFrame(features)
dataset.head()

,0,1,2,3,4,5,6
0,0.0,2.0,4.0,118.45,140.581724,0.010045,0.027637
1,1.0,4.0,4.0,112.03,86.312089,0.018072,0.044742
2,2.0,4.0,4.0,114.56,182.717163,0.015291,0.069932
3,2.0,3.0,4.0,119.92,117.964885,0.014758,0.071098
4,2.0,4.0,4.0,117.35,134.18937,0.018822,0.05269


In [145]:


def print_features(df):
    # Check chromagram feature values
    features_pauses = df.loc[:,0]
    pauses_min = features_pauses.min()
    pauses_max = features_pauses.max()
    # stack all features into a single series so we don't get a mean of means or stdev of stdevs
    pauses_mean = features_pauses.mean()
    pauses_stdev = features_pauses.std()
    print(f' pauses feature:       \
    min = {pauses_min:.3f}, \
    max = {pauses_max:.3f}, \
    mean = {pauses_mean:.3f}, \
    deviation = {pauses_stdev:.3f}') 

    # Check mel spectrogram feature values
    # Check chromagram feature values
    features_rate_of_speech = df.loc[:,1]
    rate_of_speech_min = features_rate_of_speech.min()
    rate_of_speech_max = features_rate_of_speech.max()
    # stack all features into a single series so we don't get a mean of means or stdev of stdevs
    rate_of_speech_mean = features_rate_of_speech.mean()
    rate_of_speech_stdev = features_rate_of_speech.std()
    print(f' rate_of_speech feature:       \
    min = {rate_of_speech_min:.3f}, \
    max = {rate_of_speech_max:.3f}, \
    mean = {rate_of_speech_mean:.3f}, \
    deviation = {rate_of_speech_stdev:.3f}') 

    # Check chromagram feature values
    features_articulation_rate = df.loc[:,2]
    articulation_rate_min = features_articulation_rate.min()
    articulation_rate_max = features_articulation_rate.max()
    # stack all features into a single series so we don't get a mean of means or stdev of stdevs
    articulation_rate_mean = features_articulation_rate.mean()
    articulation_rate_stdev = features_articulation_rate.std()
    print(f' articulation_rate feature:       \
    min = {articulation_rate_min:.3f}, \
    max = {articulation_rate_max:.3f}, \
    mean = {articulation_rate_mean:.3f}, \
    deviation = {articulation_rate_stdev:.3f}') 
    
    
    features_f0_mean_praat = df.loc[:,3]
    f0_mean_min = features_f0_mean_praat.min()
    f0_mean_max = features_f0_mean_praat.max()
    # stack all features into a single series so we don't get a mean of means or stdev of stdevs
    f0_mean_mean = features_f0_mean_praat.mean()
    f0_mean_stdev = features_f0_mean_praat.std()
    print(f' f0_mean_stdev feature:       \
    min = {f0_mean_min:.3f}, \
    max = {f0_mean_max:.3f}, \
    mean = {f0_mean_mean:.3f}, \
    deviation = {f0_mean_stdev:.3f}') 
    


    
    features_rms = df.loc[:,4]
    rms_min = features_rms.min()
    rms_max = features_rms.max()
    # stack all features into a single series so we don't get a mean of means or stdev of stdevs
    rms_mean = features_rms.mean()
    rms_stdev = features_rms.std()
    print(f' rms feature:       \
    min = {rms_min:.3f}, \
    max = {rms_max:.3f}, \
    mean = {rms_mean:.3f}, \
    deviation = {rms_stdev:.3f}') 
    
    features_zcr = df.loc[:,5]
    zcr_min = features_zcr.min()
    zcr_max = features_zcr.max()
    # stack all features into a single series so we don't get a mean of means or stdev of stdevs
    zcr_mean = features_zcr.mean()
    zcr_stdev = features_zcr.std()
    print(f' zcr  feature:       \
    min = {zcr_min:.3f}, \
    max = {zcr_max:.3f}, \
    mean = {zcr_mean:.3f}, \
    deviation = {zcr_stdev:.3f}') 
    
    
print_features(features_df)

 pauses feature:           min = 0.000,     max = 5.000,     mean = 1.778,     deviation = 1.133
 rate_of_speech feature:           min = 1.000,     max = 5.000,     mean = 2.764,     deviation = 0.984
 articulation_rate feature:           min = 1.000,     max = 6.000,     mean = 4.151,     deviation = 0.728
 f0_mean_stdev feature:           min = 95.150,     max = 272.880,     mean = 174.029,     deviation = 50.443
 rms feature:           min = 0.002,     max = 0.051,     mean = 0.013,     deviation = 0.008
 zcr  feature:           min = 0.016,     max = 0.117,     mean = 0.068,     deviation = 0.019


In [138]:
features_pauses = dataset.loc[:,6]

In [135]:
features_f0mean_librosa = dataset.loc[:4]

In [136]:
features_f0mean_librosa

,0,1,2,3,4,5,6
0,0.0,2.0,4.0,118.45,140.581724,0.010045,0.027637
1,1.0,4.0,4.0,112.03,86.312089,0.018072,0.044742
2,2.0,4.0,4.0,114.56,182.717163,0.015291,0.069932
3,2.0,3.0,4.0,119.92,117.964885,0.014758,0.071098
4,2.0,4.0,4.0,117.35,134.18937,0.018822,0.05269


In [ ]:
b = numpy.delete(a, indices, axis=0)

In [139]:
f=features

In [146]:
features=features[:,[0,1,2,3,5,6]]

In [148]:
first_three_features=features[:,[0,1,2]]

In [150]:
first_three_features

array([[0.0, 2.0, 4.0],
       [1.0, 4.0, 4.0],
       [2.0, 4.0, 4.0],
       ...,
       [2.0, 4.0, 4.0],
       [1.0, 4.0, 5.0],
       [1.0, 2.0, 4.0]], dtype=object)

In [149]:
last_three_features=features[:,[3,4,5]]

In [151]:
last_three_features

array([[118.45, 0.010045279748737812, 0.027637313308599695],
       [112.03, 0.018072206526994705, 0.044742021023592086],
       [114.56, 0.015291419811546803, 0.06993198249619482],
       ...,
       [120.06, 0.0212356299161911, 0.05797243626331811],
       [117.49, 0.01753460429608822, 0.06474148592085235],
       [110.86, 0.009194008074700832, 0.0486066519216134]], dtype=object)

In [160]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = StandardScaler()
# keep our unscaled features just in case we need to process them alternatively
last_three_features_scaled = last_three_features 
last_three_features_scaled = scaler.fit_transform(last_three_features_scaled)

scaler = MinMaxScaler()
# keep our unscaled features just in case we need to process them alternatively
last_three_feature_features_minmax = last_three_features
features_minmax = scaler.fit_transform(features_minmax)

In [163]:
features_scaled = np.append(first_three_features, last_three_features_scaled, axis=1)

In [164]:
features_minmax = np.append(first_three_features, features_minmax, axis=1)

In [200]:
lables1.shape[0]

694

In [166]:
print('\033[1m'+'Standard Scaling:\n'+'\033[0m')
features_scaled_df = pd.DataFrame(features_scaled)
print_features(features_scaled_df)

print('\n\n\033[1m'+'MinMax Scaling:\n'+'\033[0m')
features_minmax_df = pd.DataFrame(features_minmax)
print_features(features_minmax_df)

Standard Scaling:

 pauses feature:           min = 0.000,     max = 5.000,     mean = 1.778,     deviation = 1.133
 rate_of_speech feature:           min = 1.000,     max = 5.000,     mean = 2.764,     deviation = 0.984
 articulation_rate feature:           min = 1.000,     max = 6.000,     mean = 4.151,     deviation = 0.728
 f0_mean_stdev feature:           min = -1.565,     max = 1.961,     mean = -0.000,     deviation = 1.001
 rms feature:           min = -1.382,     max = 4.984,     mean = -0.000,     deviation = 1.001
 zcr  feature:           min = -2.694,     max = 2.585,     mean = 0.000,     deviation = 1.001


MinMax Scaling:

 pauses feature:           min = 0.000,     max = 5.000,     mean = 1.778,     deviation = 1.133
 rate_of_speech feature:           min = 1.000,     max = 5.000,     mean = 2.764,     deviation = 0.984
 articulation_rate feature:           min = 1.000,     max = 6.000,     mean = 4.151,     deviation = 0.728
 f0_mean_stdev feature:           min = 0.00

In [183]:
from sklearn.model_selection import train_test_split
############# Unscaled test/train set #############
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    lables1, 
    test_size=0.2, 
    random_state=69
)

############ Standard Scaled test/train set ###########
# The labels/classes (y_train, y_test) never change, keep old values 
X_train_scaled, X_test_scaled, _, _ = train_test_split(
    features_scaled, 
    lables1, 
    test_size=0.2, 
    random_state=69
)

############# MinMax Scaled test/train set ###############
# The labels/classes (y_train, y_test) never change, keep old values 
X_train_minmax, X_test_minmax, _, _ = train_test_split(
    features_minmax, 
    lables1, 
    test_size=0.2, 
    random_state=69
)

In [209]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classification_models = [
    KNeighborsClassifier(),#(3),
    SVC(kernel='linear'),#, C=0.025),
    SVC(kernel='rbf'),
    DecisionTreeClassifier(),#max_depth=5),
    RandomForestClassifier(),#max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

scores = []
for model in classification_models:
    model.fit(X_train, y_train)
    score = model.score(X_test_scaled, y_test)
    model_name = type(model).__name__
    if model_name=='SVC' and model.kernel=='rbf': model_name+=' RBF kernel'
    scores.append((model_name,(f'{100*score:.2f}%')))
# Make it pretty
scores_df = pd.DataFrame(scores,columns=['Classifier','Accuracy Score'])
scores_df.sort_values(by='Accuracy Score',axis=0,ascending=False)

,Classifier,Accuracy Score
3,DecisionTreeClassifier,53.96%
4,RandomForestClassifier,49.64%
0,KNeighborsClassifier,48.92%
2,SVC RBF kernel,48.92%
5,AdaBoostClassifier,48.92%
6,GaussianNB,48.92%
7,QuadraticDiscriminantAnalysis,45.32%
1,SVC,34.53%


In [210]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classification_models = [
    KNeighborsClassifier(),#(3),
    SVC(kernel='linear'),#, C=0.025),
    SVC(kernel='rbf'),
    DecisionTreeClassifier(),#max_depth=5),
    RandomForestClassifier(),#max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

scores = []
for model in classification_models:
    model.fit(X_train_scaled, y_train)
    score = model.score(X_test_scaled, y_test)
    model_name = type(model).__name__
    if model_name=='SVC' and model.kernel=='rbf': model_name+=' RBF kernel'
    scores.append((model_name,(f'{100*score:.2f}%')))
# Make it pretty
scores_df = pd.DataFrame(scores,columns=['Classifier','Accuracy Score'])
scores_df.sort_values(by='Accuracy Score',axis=0,ascending=False)

,Classifier,Accuracy Score
2,SVC RBF kernel,58.27%
7,QuadraticDiscriminantAnalysis,58.27%
0,KNeighborsClassifier,57.55%
6,GaussianNB,55.40%
1,SVC,54.68%
4,RandomForestClassifier,53.24%
5,AdaBoostClassifier,48.92%
3,DecisionTreeClassifier,48.20%


In [211]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classification_models = [
    KNeighborsClassifier(),#(3),
    SVC(kernel='linear'),#, C=0.025),
    SVC(kernel='rbf'),
    DecisionTreeClassifier(),#max_depth=5),
    RandomForestClassifier(),#max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

scores = []
for model in classification_models:
    model.fit(X_train_minmax, y_train)
    score = model.score(X_test_minmax, y_test)
    model_name = type(model).__name__
    if model_name=='SVC' and model.kernel=='rbf': model_name+=' RBF kernel'
    scores.append((model_name,(f'{100*score:.2f}%')))
# Make it pretty
scores_df = pd.DataFrame(scores,columns=['Classifier','Accuracy Score'])
scores_df.sort_values(by='Accuracy Score',axis=0,ascending=False)

,Classifier,Accuracy Score
7,QuadraticDiscriminantAnalysis,58.27%
2,SVC RBF kernel,57.55%
1,SVC,56.83%
4,RandomForestClassifier,56.12%
6,GaussianNB,55.40%
0,KNeighborsClassifier,52.52%
5,AdaBoostClassifier,48.92%
3,DecisionTreeClassifier,46.76%


In [214]:
#################### minmax scaled #######################
# Rebuild the training set from the minmax scaled features
# The labels/classes (y_train, y_test) never change, keep old values 
X_train_minmax, X_test_minmax, _, _ = train_test_split(
    features_minmax, 
    lables1, 
    test_size=0.2, 
    random_state=69
)

# Fit the model again on the minmax scaled features
model.fit(X_train_minmax, y_train)

print(f'MinMax scaled MLP Model\'s accuracy on training set is {100*model.score(X_train_minmax, y_train):.2f}%')
print(f'MinMax sacled MLP Model\'s accuracy on test set is {100*model.score(X_test_minmax, y_test):.2f}%\n')

#################### standard scaled #######################
# Rebuild the training set from the standard scaled features
# The labels/classes (y_train, y_test) never change, keep old values 
X_train_scaled, X_test_scaled, _, _ = train_test_split(
    features_scaled, 
    lables1, 
    test_size=0.2, 
    random_state=69
)
# Fit the model again on the scaled features
model.fit(X_train_scaled, y_train)

print(f'Standard scaled MLP Model\'s accuracy on training set is {100*model.score(X_train_scaled, y_train):.2f}%')
print(f'Standard scaled MLP Model\'s accuracy on test set is {100*model.score(X_test_scaled, y_test):.2f}%')

MinMax scaled MLP Model's accuracy on training set is 56.76%
MinMax sacled MLP Model's accuracy on test set is 58.27%

Standard scaled MLP Model's accuracy on training set is 56.76%
Standard scaled MLP Model's accuracy on test set is 58.27%


In [216]:
from sklearn.neural_network import MLPClassifier

# Default 'off-the-shelf' MLP from sklearn
model = MLPClassifier(
    random_state = 69
)

#################### unscaled features #####################
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    lables1, 
    test_size=0.2, 
    random_state=69
)

model.fit(X_train, y_train)

print(f'Possible emotions predicted by model:{model.classes_}')
print(f'Unscaled MLP Model\'s accuracy on training set is {100*model.score(X_train, y_train):.2f}%')
print(f'Unscaled MLP Model\'s accuracy on test set is {100*model.score(X_test, y_test):.2f}%')

Possible emotions predicted by model:['1' '2' '7']
Unscaled MLP Model's accuracy on training set is 51.53%
Unscaled MLP Model's accuracy on test set is 54.68%


In [217]:
#################### minmax scaled #######################
# Rebuild the training set from the minmax scaled features
# The labels/classes (y_train, y_test) never change, keep old values 
X_train_minmax, X_test_minmax, _, _ = train_test_split(
    features_minmax, 
    lables1, 
    test_size=0.2, 
    random_state=69
)

# Fit the model again on the minmax scaled features
model.fit(X_train_minmax, y_train)

print(f'MinMax scaled MLP Model\'s accuracy on training set is {100*model.score(X_train_minmax, y_train):.2f}%')
print(f'MinMax sacled MLP Model\'s accuracy on test set is {100*model.score(X_test_minmax, y_test):.2f}%\n')

#################### standard scaled #######################
# Rebuild the training set from the standard scaled features
# The labels/classes (y_train, y_test) never change, keep old values 
X_train_scaled, X_test_scaled, _, _ = train_test_split(
    features_scaled, 
    lables1, 
    test_size=0.2, 
    random_state=69
)
# Fit the model again on the scaled features
model.fit(X_train_scaled, y_train)

print(f'Standard scaled MLP Model\'s accuracy on training set is {100*model.score(X_train_scaled, y_train):.2f}%')
print(f'Standard scaled MLP Model\'s accuracy on test set is {100*model.score(X_test_scaled, y_test):.2f}%')

C:\Users\kheder yazgi\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MinMax scaled MLP Model's accuracy on training set is 56.22%
MinMax sacled MLP Model's accuracy on test set is 59.71%

Standard scaled MLP Model's accuracy on training set is 62.70%
Standard scaled MLP Model's accuracy on test set is 54.68%


C:\Users\kheder yazgi\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [218]:
from sklearn.model_selection import GridSearchCV

# Now that we know standard scaling is best for our features, we'll use those for our training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    features_scaled, 
    lables1, 
    test_size=0.2, 
    random_state=69
)

# Initialize the MLP Classifier and choose parameters we want to keep constant
model = MLPClassifier(
    # tune batch size later 
    batch_size=256,  
    # keep random state constant to accurately compare subsequent models
    random_state=69
)

# Choose the grid of hyperparameters we want to use for Grid Search to build our candidate models
parameter_space = {
    # A single hidden layer of size between 8 (output classes) and 180 (input features) neurons is most probable
    # It's a bad idea at guessing the number of hidden layers to have
    # ...but we'll give 2 and 3 hidden layers a shot to reaffirm our suspicions that 1 is best
    'hidden_layer_sizes': [(8,), (180,), (300,),(100,50,),(10,10,10)], 
    'activation': ['tanh','relu', 'logistic'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.001, 0.01],
    'epsilon': [1e-08, 0.1 ],
    'learning_rate': ['adaptive', 'constant']
}
   
# Create a grid search object which will store the scores and hyperparameters of all candidate models 
grid = GridSearchCV(
    model, 
    parameter_space,
    cv=10,
    n_jobs=4)
# Fit the models specified by the parameter grid 
grid.fit(X_train, y_train)

# get the best hyperparameters from grid search object with its best_params_ attribute
print('Best parameters found:\n', grid.best_params_)

Best parameters found:
 {'activation': 'tanh', 'alpha': 0.0001, 'epsilon': 1e-08, 'hidden_layer_sizes': (300,), 'learning_rate': 'adaptive', 'solver': 'adam'}


C:\Users\kheder yazgi\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [221]:
# declare again so I don't have to re-run gridsearch when I restart the jupyter kernel
# our default X_train, X_test are now standard scaled features since we know they're best
X_train, X_test, y_train, y_test = train_test_split(
    features_scaled, 
    lables1, 
    test_size=0.2, 
    random_state=69
)

model = MLPClassifier(
    activation='logistic', 
    solver='adam', 
    alpha=0.001, 
    beta_1=0.9,
    beta_2=0.999,
    batch_size=256, 
    epsilon=1e-08, 
    hidden_layer_sizes=(300,), 
    learning_rate='adaptive',
    max_iter=1000, # I've found for this task, loss converges at ~1000 iterations
    random_state=69,
)

model.fit(X_train, y_train)

print(f'MLP Model\'s accuracy on training set is {100*model.score(X_train, y_train):.2f}%')
print(f'MLP Model\'s accuracy on test set is {100*model.score(X_test, y_test):.2f}%')

MLP Model's accuracy on training set is 55.68%
MLP Model's accuracy on test set is 53.96%
